In [1]:
import pandas
import json

### Tuve que instalar google-cloud-storage y google cloud language


from google.cloud import storage

import os
from google.cloud import language_v1

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='bubbly-realm-344409-7b7b235b215d.json'
client = language_v1.LanguageServiceClient()

In [8]:
def read_file_blob(bucket_name,destination_blob_name):
    """Read a file from the bucket."""
 
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
 
    # read as string
    read_output = blob.download_as_string().decode("utf8")

    return read_output

In [9]:
read_file_blob('ejemplo_nlp', 'prueba_es.txt')

'BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota informativa Proyecciones macroeconómicas de la economía española (2021-2023) Esta nota describe los rasgos principales de las proyecciones macroeconómicas de la economía española para el período 2021-2023, que el Banco de España publica hoy en su página web (disponibles en este enlace). En relación a las proyecciones publicadas el pasado diciembre, las actuales incorporan la nueva información conocida desde entonces, lo que incluye, entre otros elementos, las estimaciones preliminares de la Contabilidad Nacional Trimestral para el cuarto trimestre de 2020 y los cambios observados en los supuestos técnicos que subyacen a la construcción de los escenarios. Las perspectivas económicas siguen sujetas a un elevado grado de incertidumbre, vinculada tanto, en el corto plazo, a la evolución de la pandemia y la velocidad del proceso de inmunización de la población como, a medio plazo, a las consecuencias económicas de la crisis. A su vez, estas últim

In [3]:
def classify(gcs_content_uri, verbose = True):
    """Classify the input text into categories. """

    language_client = language_v1.LanguageServiceClient()
    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type_": type_, "language": language}
   
    response = language_client.classify_text(request={"document": document})
    categories = response.categories

    result = {}

    for category in categories:
        # Turn the categories into a dictionary of the form:
        # {category.name: category.confidence}, so that they can
        # be treated as a sparse vector.
        result[category.name] = category.confidence

    if verbose:
        print(gcs_content_uri)
        for category in categories:
            print(u"=" * 20)
            print(u"{:<16}: {}".format("category", category.name))
            print(u"{:<16}: {}".format("confidence", category.confidence))

    return result

In [4]:
response_classify = classify("gs://ejemplo_nlp/prueba_es.txt")

gs://ejemplo_nlp/prueba_es.txt


In [5]:
response_classify

{}

In [8]:
def sample_classify_text(gcs_content_uri):
    """
    Classifying Content in text file stored in Cloud Storage

    Args:
      gcs_content_uri Google Cloud Storage URI where the file content is located.
      e.g. gs://[Your Bucket]/[Path to File]
      The text file must include at least 20 words.
    """

    client = language_v1.LanguageServiceClient()

    # gcs_content_uri = 'gs://cloud-samples-data/language/classify-entertainment.txt'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type_": type_, "language": language}

    response = client.classify_text(request = {'document': document})
    
    category_result = []
    
    for category in response.categories:
        category_dic = {"id":"1234567"}
        category_dic['Category_name'] = category.name
        category_dic['Category_confidence'] = category.confidence

        category_result.append(category_dic)    

    return  category_result


In [9]:
sample_classify_text("gs://ejemplo_nlp/prueba_es.txt")

[]

In [10]:
def analyze_text_syntax(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

    response = client.analyze_syntax(document=document)

    fmts = "{:10}: {}"
    print(fmts.format("sentences", len(response.sentences)))
    print(fmts.format("tokens", len(response.tokens)))
    for token in response.tokens:
        print(fmts.format(token.part_of_speech.tag.name, token.text.content))

In [11]:
analyze_text_syntax(read_file_blob('ejemplo_nlp', 'prueba_es.txt'))

sentences : 26
tokens    : 1119
NOUN      : BANCO
ADP       : DE
NOUN      : ESPAÑA
NOUN      : Eurosistema
NUM       : 23.03.2021
NOUN      : Nota
ADJ       : informativa
NOUN      : Proyecciones
ADJ       : macroeconómicas
ADP       : de
DET       : la
NOUN      : economía
ADJ       : española
PUNCT     : (
NUM       : 2021-2023
PUNCT     : )
DET       : Esta
NOUN      : nota
VERB      : describe
DET       : los
NOUN      : rasgos
ADJ       : principales
ADP       : de
DET       : las
NOUN      : proyecciones
ADJ       : macroeconómicas
ADP       : de
DET       : la
NOUN      : economía
ADJ       : española
ADP       : para
DET       : el
NOUN      : período
NUM       : 2021-2023
PUNCT     : ,
ADP       : que
DET       : el
NOUN      : Banco
ADP       : de
NOUN      : España
VERB      : publica
ADV       : hoy
ADP       : en
PRON      : su
NOUN      : página
NOUN      : web
PUNCT     : (
ADJ       : disponibles
ADP       : en
DET       : este
NOUN      : enlace
PUNCT     : )
PUNCT   

In [17]:
def analyze_text_sentiment(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

    response = client.analyze_sentiment(document=document)

    sentiment = response.document_sentiment
    results = dict(
        text=text,
        score=f"{sentiment.score:.1%}",
        magnitude=f"{sentiment.magnitude:.1%}",
    )
    for k, v in results.items():
        print(f"{k:10}: {v}")

In [18]:
analyze_text_sentiment(read_file_blob('ejemplo_nlp', 'prueba_es.txt'))

text      : BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota informativa Proyecciones macroeconómicas de la economía española (2021-2023) Esta nota describe los rasgos principales de las proyecciones macroeconómicas de la economía española para el período 2021-2023, que el Banco de España publica hoy en su página web (disponibles en este enlace). En relación a las proyecciones publicadas el pasado diciembre, las actuales incorporan la nueva información conocida desde entonces, lo que incluye, entre otros elementos, las estimaciones preliminares de la Contabilidad Nacional Trimestral para el cuarto trimestre de 2020 y los cambios observados en los supuestos técnicos que subyacen a la construcción de los escenarios. Las perspectivas económicas siguen sujetas a un elevado grado de incertidumbre, vinculada tanto, en el corto plazo, a la evolución de la pandemia y la velocidad del proceso de inmunización de la población como, a medio plazo, a las consecuencias económicas de la crisis. A su vez, 

In [20]:
def analyze_text_entities(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

    response = client.analyze_entities(document=document)

    for entity in response.entities:
        print("=" * 80)
        results = dict(
            name=entity.name,
            type=entity.type_.name,
            salience=f"{entity.salience:.1%}",
            wikipedia_url=entity.metadata.get("wikipedia_url", "-"),
            mid=entity.metadata.get("mid", "-"),
        )
        for k, v in results.items():
            print(f"{k:15}: {v}")

In [21]:
analyze_text_entities(read_file_blob('ejemplo_nlp', 'prueba_es.txt'))

name           : crisis
type           : EVENT
salience       : 18.0%
wikipedia_url  : -
mid            : -
name           : escenario
type           : LOCATION
salience       : 9.2%
wikipedia_url  : -
mid            : -
name           : proceso
type           : EVENT
salience       : 5.1%
wikipedia_url  : -
mid            : -
name           : grado
type           : OTHER
salience       : 4.8%
wikipedia_url  : -
mid            : -
name           : media
type           : OTHER
salience       : 4.3%
wikipedia_url  : -
mid            : -
name           : pandemia
type           : OTHER
salience       : 3.7%
wikipedia_url  : -
mid            : -
name           : cambios
type           : OTHER
salience       : 3.5%
wikipedia_url  : -
mid            : -
name           : agentes
type           : PERSON
salience       : 3.4%
wikipedia_url  : -
mid            : -
name           : Banco de España
type           : ORGANIZATION
salience       : 2.7%
wikipedia_url  : -
mid            : /g/11ckkwl4f

type           : OTHER
salience       : 0.1%
wikipedia_url  : -
mid            : -
name           : recuperación
type           : EVENT
salience       : 0.1%
wikipedia_url  : -
mid            : -
name           : crecimiento
type           : OTHER
salience       : 0.1%
wikipedia_url  : -
mid            : -
name           : formulación
type           : OTHER
salience       : 0.1%
wikipedia_url  : -
mid            : -
name           : progresión
type           : EVENT
salience       : 0.0%
wikipedia_url  : -
mid            : -
name           : actividad
type           : OTHER
salience       : 0.0%
wikipedia_url  : -
mid            : -
name           : central
type           : PERSON
salience       : 0.0%
wikipedia_url  : -
mid            : -
name           : resolución
type           : OTHER
salience       : 0.0%
wikipedia_url  : -
mid            : -
name           : resultado
type           : OTHER
salience       : 0.0%
wikipedia_url  : -
mid            : -
name           : variación
ty